## 객체 탐지(Object Detection)을 통한 거리뷰 이미지 위험도 분석

이 노트북에서는 YOLO(You Only Look Once) v8 물체 탐지 모델을 활용하여 거리뷰 이미지에서 노상에 적치된 객체와 협소도로 주차 문제를 분석합니다. </br> 분석 결과를 엑셀에 기록하여 자치구별로 정리하며, 이 데이터를 바탕으로 자치구별 위험도를 평가하려 합니다.

#### 1. ultralytics 및 기타 패키지 설치하기

[ultralytics](https://github.com/ultralytics/ultralytics)는 YOLO v8 모델의 사전 훈련된 버전을 제공하며, 사용자가 간편하게 추론을 수행할 수 있는 패키지입니다.

In [ ]:
pip show XlsxWriter

In [ ]:
pip install -r requirements.txt

#### 2. 패키지 불러오기

In [ ]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
from ultralytics import YOLO

from common import imread, imwrite, save_dataframe_to_excel
from detect import load_yolo_model, save_results_to_image

#### 3. 사전 학습 모델 불러오기

In [ ]:
seg_model = load_yolo_model('yolov8n-seg.pt')

#### 4. 탐지할 객체 목록 불러오기

In [ ]:
with open('classes.json', 'r') as file:
    classes = json.load(file)

# The classes dictionary values will be the column names
columns = list(classes.values())

#### 5. 거리뷰 이미지 데이터셋 불러오기

In [ ]:
# 분석하고자 하는 이미지 폴더 dir_path에 설정
dir_path = 'dataset/강동/'

gu = dir_path.split('/')[-2]

try:
    if not os.path.exists(dir_path):
        raise ValueError(f'The directory {dir_path} does not exist.')
    file_lists = os.listdir(dir_path)
    
    if len(file_lists) == 0:
        raise ValueError(f'Empty folder')

except ValueError as e:
    print(e)
    print("Stopping notebook execution due to the above error.")

print('---'*10)
print(f'Folder path : {dir_path}\nNumber of images : {len(file_lists)}')

#### 6. 추론 결과를 저장할 데이터시트 생성

In [ ]:
df = pd.DataFrame(columns=columns)
df.insert(0, 'file', [])

#### 7. YOLO v8 사전 학습 모델을 활용해 추론

In [ ]:
# Prdict
output_dirs = 'output_'+gu
if not os.path.exists(output_dirs):
    os.makedirs(output_dirs)
    
for f in file_lists:
    img = imread(os.path.join(dir_path, f))
    results = seg_model.predict(img, device=0, conf=0.3)
    
    data = {class_name: 0 for class_name in columns}
     
    data['file'] = f

    if results:
        boxes = results[0].boxes
        class_id = boxes.cls
        
        for i in class_id:
            id = int(i.tolist())
            try: 
                data[classes[str(id)]] += 1
            except KeyError:
                continue
        
        # (선택사항) 결과 이미지 저장
        save_results_to_image(results, os.path.join(output_dirs, f))
            
    df.loc[len(df)] = data

#### 8. 추론 결과를 동일 자치구별 합산

In [ ]:
# Split the 'file' column by 'PIC' and store the first part in the 'road' column
df['road'] = df['file'].str.split('_').str[0]

# Drop the 'file' column
df.drop('file', axis=1, inplace=True)

agg_dict = {col: 'sum' for col in columns}

# Group by 'road' and sum the values in the 'person' column, then reset the index
df_combined = df.groupby('road').agg(agg_dict).reset_index()

df_combined.head()

#### 9. 엑셀 데이터시트로 저장

In [ ]:
# Save the results to excel 
save_dataframe_to_excel(df_combined, f'outputs_{gu}.xlsx')

#### Reference 
- YOLO v8 : https://github.com/ultralytics/ultralytics